In [2]:
import requests
import pandas as pd
from datetime import date
import numpy as np
import time as tt

# URL template (corrected)
url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15"

# Instrument tokens
tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

# Interval mapping
KI = {
    "3m": "3minute",
    "5m": "5minute",
    "15m": "15minute",
    "60m": "60minute",
    "day": "day",
}

# Function to get candle data
def getCandles(symbol, fromDate, toDate, timeframe):
    headers = {
        "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
        # Replace this with your actual token securely
    }
    curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
    print("Requesting:", curUrl)

    try:
        session = requests.session()
        response = session.get(curUrl, headers=headers)
        r = response.json()
    except Exception as error:
        print("API request failed:", error)
        return pd.DataFrame()

    try:
        history = r["data"]["candles"]
        history_df = pd.DataFrame(history)
        history_df = history_df.rename(
            columns={
                0: "Datetime",
                1: "open",
                2: "high",
                3: "low",
                4: "close",
                5: "volume",
            }
        )
        history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
        history_df.set_index("Datetime", inplace=True)
        return history_df
    except Exception as error:
        print("Data processing failed:", error)
        return pd.DataFrame()


def StochRSI(df, series, period=14, smoothK=3, smoothD=3):
    delta = series.diff()
    ups = delta.clip(lower=0)
    downs = -delta.clip(upper=0)

    avg_gain = ups.rolling(window=period).mean()
    avg_loss = downs.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Calculate StochRSI
    min_rsi = rsi.rolling(window=period).min()
    max_rsi = rsi.rolling(window=period).max()
    stochrsi = (rsi - min_rsi) / (max_rsi - min_rsi)

    stochrsi_K = stochrsi.rolling(window=smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(window=smoothD).mean()

    df = df.copy()
    df["SRSI"] = stochrsi
    df["K"] = stochrsi_K
    df["D"] = stochrsi_D

    return df


# Example usage
minute5data = getCandles("NIFTY", "2025-06-11", "2025-06-15", KI["5m"])
if not minute5data.empty:
    sc = StochRSI(minute5data, minute5data["close"])
    print(sc[["close", "K", "D", "SRSI"]].tail(5))
else:
    print("No data returned.")




Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15
                              close         K         D      SRSI
Datetime                                                         
2025-06-13 15:05:00+05:30  24727.90  0.425454  0.197493  0.775280
2025-06-13 15:10:00+05:30  24725.60  0.659608  0.417363  0.702465
2025-06-13 15:15:00+05:30  24717.95  0.712040  0.599034  0.658376
2025-06-13 15:20:00+05:30  24729.35  0.738914  0.703521  0.855902
2025-06-13 15:25:00+05:30  24735.55  0.824377  0.758444  0.958854
